In [31]:
from Bio import Entrez
import pandas as pd
import time
from docx import Document
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
from langchain.prompts import ChatPromptTemplate
import re

load_dotenv()
chat = ChatOpenAI(model='gpt-4o', temperature=0.2, max_tokens=4096)  # controle de max_tokens conforme necessidade
Entrez.email = ''

In [32]:
def search(query, retmax=30):  retmax: limite de artigos por busca
    handle = Entrez.esearch(db='pubmed', sort='relevance', retmax=retmax, retmode='xml', term=query)
    results = Entrez.read(handle)
    return results['IdList']

In [33]:
def fetch_details(id_list):
    ids = ','.join(id_list)
    handle = Entrez.efetch(db='pubmed', retmode='xml', id=ids)
    results = Entrez.read(handle)
    return results

In [34]:
topics = [
    'Whey Protein adverse effects',
    'Whey Protein interactions',
    'Whey Protein contraindications',
    'Whey Protein clinical applications',
    'Whey Protein safe doses',
    'Whey Protein pregnancy lactation pediatric'
]


articles_list = []

for topic in topics:
    print(f'Buscando artigos para: {topic}')
    studiesIdList = search(topic, retmax=100)  # Limite de artigos por tópico(Ajuste conforme necessidade)
    print(f'  {len(studiesIdList)} artigos encontrados.')
    if not studiesIdList:
        continue
    chunk_size = 100
    for i in range(0, len(studiesIdList), chunk_size):
        chunk = studiesIdList[i:i+chunk_size]
        papers = fetch_details(chunk)
        for paper in papers['PubmedArticle']:
            pmid = paper['MedlineCitation']['PMID']
            title = paper['MedlineCitation']['Article'].get('ArticleTitle', 'No Title')
            try:
                abstract = ' '.join(paper['MedlineCitation']['Article']['Abstract']['AbstractText'])
            except:
                abstract = 'No Abstract'
            try:
                authors = paper['MedlineCitation']['Article']['AuthorList']
                authors = '; '.join([f"{a.get('LastName','')} {a.get('Initials','')}".strip() for a in authors if 'LastName' in a])
            except:
                authors = ''
            journal = paper['MedlineCitation']['Article']['Journal'].get('Title', '')
            language = paper['MedlineCitation']['Article'].get('Language', [''])[0]
            try:
                year = paper['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate'].get('Year', '')
                month = paper['MedlineCitation']['Article']['Journal']['JournalIssue']['PubDate'].get('Month', '')
            except:
                year = ''
                month = ''
            doi = ''
            try:
                for id_item in paper['MedlineCitation']['Article'].get('ELocationID', []):
                    if id_item.attributes['EIdType'] == 'doi':
                        doi = str(id_item)
                        break
            except:
                pass

            article_data = {
                'Topico_busca': topic,
                'PMID': pmid,
                'Title': title,
                'Abstract': abstract,
                'Authors': authors,
                'Journal': journal,
                'Language': language,
                'Year': year,
                'Month': month,
                'DOI': doi,
            }
            articles_list.append(article_data)
        time.sleep(1)

df = pd.DataFrame(articles_list)
df.to_excel('Resultados_PubMed_WheyProtein.xlsx', index=False)
print("Arquivo Excel salvo!")

Buscando artigos para: Whey Protein adverse effects
  100 artigos encontrados.
Buscando artigos para: Whey Protein interactions
  100 artigos encontrados.
Buscando artigos para: Whey Protein contraindications
  10 artigos encontrados.
Buscando artigos para: Whey Protein clinical applications
  93 artigos encontrados.
Buscando artigos para: Whey Protein safe doses
  10 artigos encontrados.
Buscando artigos para: Whey Protein pregnancy lactation pediatric
  19 artigos encontrados.
Arquivo Excel salvo!


In [35]:
# Agrupa os artigos por tópicos do modelo
topic_keywords = {
    "DESCRIÇÃO GERAL": ["whey protein", "proteína do soro", "general", "overview"],
    "MECANISMOS DE AÇÃO": ["mechanism", "mecanismo", "pathway", "metabolic", "biochemical"],
    "BENEFÍCIOS E APLICAÇÕES": ["benefit", "aplicação", "improvement", "performance", "muscle", "clinical", "tratamento", "disease", "elderly", "health", "athlete"],
    "DOSES USUAIS": ["dose", "dosing", "posology", "mg", "g/kg", "ingestão", "consumo"],
    "EFEITOS COLATERAIS": ["side effect", "efeito colateral", "adverse", "toxicity", "tóxico"],
    "TOXICIDADE E SEGURANÇA": ["safety", "toxicidade", "safe", "security", "toxic", "segurança"],
    "USO EM POPULAÇÕES ESPECIAIS": ["pregnancy", "lactation", "gestante", "lactante", "pediatria", "child", "infant", "elderly", "idoso"],
    "CONTRAINDICAÇÕES E PRECAUÇÕES": ["contraindication", "contraindicação", "caution", "precaução", "not recommended"],
}

def classify_article(title, abstract):
    text = f"{str(title)} {str(abstract)}".lower()
    sections = []
    for topic, keywords in topic_keywords.items():
        for kw in keywords:
            if kw in text:
                sections.append(topic)
                break
    if not sections:
        sections = ["DESCRIÇÃO GERAL"]
    return sections

articles_by_topic = {key: [] for key in topic_keywords}
for _, row in df.iterrows():
    sections = classify_article(row['Title'], row['Abstract'])
    for topic in sections:
        ref = ""
        if row['Authors'] and row['Year']:
            ref = f"{row['Authors'].split(';')[0].strip()} et al., {row['Year']}"
        elif row['Authors']:
            ref = f"{row['Authors'].split(';')[0].strip()}"
        elif row['Year']:
            ref = f"{row['Year']}"
        artigo = f"- {row['Title']} ({ref})\n  {row['Abstract']}"
        articles_by_topic[topic].append(artigo)

In [36]:
model_sections = [
    "DESCRIÇÃO GERAL",
    "MECANISMOS DE AÇÃO",
    "BENEFÍCIOS E APLICAÇÕES",
    "DOSES USUAIS",
    "EFEITOS COLATERAIS",
    "TOXICIDADE E SEGURANÇA",
    "USO EM POPULAÇÕES ESPECIAIS",
    "CONTRAINDICAÇÕES E PRECAUÇÕES",
]

In [37]:
def mega_prompt_otimizado(sec, artigos, tema="Whey Protein"):
    return f"""
Você é um redator científico especializado em redação acadêmica, com experiência na produção de artigos científicos para periódicos de alto impacto. Sua tarefa é elaborar a seção abaixo do artigo científico sobre {tema}, utilizando EXCLUSIVAMENTE os dados apresentados nos artigos fornecidos.

SIGA RIGOROSAMENTE AS INSTRUÇÕES:
- NÃO utilize nenhum conhecimento prévio, apenas os dados fornecidos.
- NÃO invente, assuma ou complemente informações que não estejam expressas nos artigos abaixo.
- NÃO repita informações desnecessárias.
- Cite os autores e o ano do estudo ao mencionar qualquer dado (exemplo: Silva et al., 2023).
- Utilize português formal, técnico e acadêmico, com variações de conectivos e estrutura textual fluida.
- Utilize marcadores, listas e destaque em negrito para usos, doses, efeitos, aplicações clínicas e outros dados quantitativos quando pertinente.
- Estruture a seção conforme exemplos a seguir:

EXEMPLO DE FORMATO DE PARÁGRAFO E CITAÇÃO:
- “A suplementação com whey protein demonstrou impacto positivo na recuperação muscular (Souza et al., 2023). Além disso, observou-se redução significativa de marcadores inflamatórios em atletas submetidos a treinamento intenso (Silva et al., 2022).”
- “**Usos específicos:**”
    • **Desempenho esportivo:** 10-30 g/dia (Carvalho et al., 2022)

SEÇÃO: {sec.upper()}

ARTIGOS FORNECIDOS:
{chr(10).join(artigos)}

Oriente-se pelo modelo abaixo ao redigir sua resposta:
- Parágrafo de abertura: contextualize brevemente o tema da seção, utilizando dados dos artigos.
- Desenvolvimento: organize as evidências em subtópicos, listas, bullets e destaque dados relevantes em negrito. Conecte ideias usando conectivos variados (além disso, dessa forma, consequentemente, etc).
- Citações obrigatórias: sempre relacione cada afirmação a pelo menos um artigo/autor citado.
- Conclusão curta da seção: feche a seção reafirmando as principais descobertas, sempre com base nos artigos.

IMPORTANTE:
- Nunca escreva referências bibliográficas ao final da seção.
- Jamais “encha linguiça”. Mantenha o texto denso, claro, objetivo, informativo e profissional.
- Não repita frases dos artigos, sempre parafraseie e organize.
"""

In [38]:
def gerar_texto_secao_bloco(sec, artigos, tema="Whey Protein", max_artigos_por_bloco=25):
    textos_finais = []
    for i in range(0, len(artigos), max_artigos_por_bloco):
        artigos_bloco = artigos[i:i + max_artigos_por_bloco]
        prompt_secao = mega_prompt_otimizado(sec, artigos_bloco, tema)
        prompt_t = ChatPromptTemplate.from_messages([
            ('system', "Você é um redator científico especializado."),
            ('user', prompt_secao)
        ])
        resposta = chat.invoke(prompt_t.format_messages())
        textos_finais.append(resposta.content)
        time.sleep(2)
    return "\n".join(textos_finais)


conteudo_artigo = {}
for sec in model_sections:
    print(f"\n--- Gerando seção: {sec} ---")
    artigos = articles_by_topic[sec]
    conteudo = gerar_texto_secao_bloco(sec, artigos, max_artigos_por_bloco=20)
    conteudo_artigo[sec] = conteudo
    time.sleep(2)


--- Gerando seção: DESCRIÇÃO GERAL ---

--- Gerando seção: MECANISMOS DE AÇÃO ---

--- Gerando seção: BENEFÍCIOS E APLICAÇÕES ---

--- Gerando seção: DOSES USUAIS ---

--- Gerando seção: EFEITOS COLATERAIS ---

--- Gerando seção: TOXICIDADE E SEGURANÇA ---

--- Gerando seção: USO EM POPULAÇÕES ESPECIAIS ---

--- Gerando seção: CONTRAINDICAÇÕES E PRECAUÇÕES ---


In [39]:
# Gera o arquivo Word final
doc = Document()
doc.add_heading('Whey Protein – Artigo Científico', 0)
for sec in model_sections:
    doc.add_heading(sec, level=1)
    doc.add_paragraph(conteudo_artigo[sec])
doc.save("Artigo_Cientifico_WheyProtein.docx")
print("Artigo gerado em Artigoooooo_Cientifico_WheyProtein.docx!")

Artigo gerado em Artigoooooo_Cientifico_WheyProtein.docx!
